In [2]:
import os
import cv2
import numpy as np
from tensorflow.keras.preprocessing.image import ImageDataGenerator
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Conv2D, MaxPooling2D, Flatten, Dense, Dropout, BatchNormalization
from tensorflow.keras.callbacks import ModelCheckpoint, ReduceLROnPlateau, EarlyStopping
from tensorflow.keras.regularizers import l2
from tensorflow.keras.optimizers import Adam
from tensorflow.keras.models import load_model
import kagglehub
from sklearn.metrics import classification_report, confusion_matrix
import seaborn as sns
import matplotlib.pyplot as plt

2025-01-08 13:32:21.402828: I external/local_xla/xla/tsl/cuda/cudart_stub.cc:32] Could not find cuda drivers on your machine, GPU will not be used.
2025-01-08 13:32:21.406776: I external/local_xla/xla/tsl/cuda/cudart_stub.cc:32] Could not find cuda drivers on your machine, GPU will not be used.
2025-01-08 13:32:21.421379: E external/local_xla/xla/stream_executor/cuda/cuda_fft.cc:477] Unable to register cuFFT factory: Attempting to register factory for plugin cuFFT when one has already been registered
E0000 00:00:1736339541.442027  449951 cuda_dnn.cc:8310] Unable to register cuDNN factory: Attempting to register factory for plugin cuDNN when one has already been registered
E0000 00:00:1736339541.447760  449951 cuda_blas.cc:1418] Unable to register cuBLAS factory: Attempting to register factory for plugin cuBLAS when one has already been registered
2025-01-08 13:32:21.468021: I tensorflow/core/platform/cpu_feature_guard.cc:210] This TensorFlow binary is optimized to use available CPU ins

In [2]:
# Télécharger le dataset ASL Alphabet
path = kagglehub.dataset_download("grassknoted/asl-alphabet")

/home/youness.ait-hadichou/Documents/5ETI/Projet_majeure/projet/lib/python3.10/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


100%|██████████| 1.03G/1.03G [00:30<00:00, 36.3MB/s]

Extracting files...


In [4]:
print("Path to dataset files:", path)

Path to dataset files: /home/youness.ait-hadichou/.cache/kagglehub/datasets/grassknoted/asl-alphabet/versions/1


In [3]:
dataset_path = "/home/youness.ait-hadichou/.cache/kagglehub/datasets/grassknoted/asl-alphabet/versions/1/asl_alphabet_train/asl_alphabet_train"

# Liste des classes
target_classes = list("ABCDEFGHIJKLMNOPQRSTUVWXYZ") + ["nothing"]

# Générateur de données 
datagen = ImageDataGenerator(
    rescale=1.0/255.0,
    validation_split=0.2,
    rotation_range=15,           
    width_shift_range=0.1,
    height_shift_range=0.1,
    zoom_range=0.1,
    shear_range=0.1,
    brightness_range=[0.9, 1.1], 
    horizontal_flip=False,       
    fill_mode='constant',
    cval=0
)

train_generator = datagen.flow_from_directory(
    dataset_path,
    target_size=(64, 64),
    batch_size=64,
    classes=target_classes,
    class_mode="sparse",
    subset="training"
)

validation_generator = datagen.flow_from_directory(
    dataset_path,
    target_size=(64, 64),
    batch_size=64,
    classes=target_classes,
    class_mode="sparse",
    subset="validation"
)

Found 64800 images belonging to 27 classes.
Found 16200 images belonging to 27 classes.


In [4]:
# Modèle
model = Sequential([
    # Premier bloc
    Conv2D(32, (3, 3), padding='same', activation="relu", 
           kernel_regularizer=l2(0.001), input_shape=(64, 64, 3)),
    BatchNormalization(),
    Conv2D(32, (3, 3), padding='same', activation="relu", 
           kernel_regularizer=l2(0.001)),
    BatchNormalization(),
    MaxPooling2D((2, 2)),
    Dropout(0.3),

    # Deuxième bloc
    Conv2D(64, (3, 3), padding='same', activation="relu", 
           kernel_regularizer=l2(0.001)),
    BatchNormalization(),
    Conv2D(64, (3, 3), padding='same', activation="relu", 
           kernel_regularizer=l2(0.001)),
    BatchNormalization(),
    MaxPooling2D((2, 2)),
    Dropout(0.3),

    # Troisième bloc
    Conv2D(128, (3, 3), padding='same', activation="relu", 
           kernel_regularizer=l2(0.001)),
    BatchNormalization(),
    MaxPooling2D((2, 2)),
    Dropout(0.3),

    # Couches denses
    Flatten(),
    Dense(256, activation="relu", kernel_regularizer=l2(0.001)),
    BatchNormalization(),
    Dropout(0.5),
    #Dense(128, activation="relu", kernel_regularizer=l2(0.0001)),
    #BatchNormalization(),
    #Dropout(0.5),
    Dense(len(target_classes), activation="softmax")
])

/home/youness.ait-hadichou/Documents/5ETI/Projet_majeure/projet/lib/python3.10/site-packages/keras/src/layers/convolutional/base_conv.py:107: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(activity_regularizer=activity_regularizer, **kwargs)
2025-01-08 13:32:42.177337: E external/local_xla/xla/stream_executor/cuda/cuda_driver.cc:152] failed call to cuInit: INTERNAL: CUDA error: Failed call to cuInit: UNKNOWN ERROR (303)


In [5]:
# Compilation
model.compile(
    optimizer=Adam(learning_rate=0.0005),  
    loss="sparse_categorical_crossentropy",
    metrics=["accuracy"]
)

# Callbacks 
checkpoint = ModelCheckpoint(
    "asl_model.keras",
    save_best_only=True,
    monitor="val_loss", 
    mode="min"
)

reduce_lr = ReduceLROnPlateau(
    monitor="val_loss",
    factor=0.2,
    patience=5,
    verbose=1,
    min_lr=1e-6
)

early_stopping = EarlyStopping(
    monitor='val_loss',
    patience=8,
    restore_best_weights=True,
    verbose=1
)

# Afficher le résumé du modèle
model.summary()

Model: "sequential"

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━┓
┃ Layer (type)                    ┃ Output Shape           ┃       Param # ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━┩
│ conv2d (Conv2D)                 │ (None, 64, 64, 32)     │           896 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ batch_normalization             │ (None, 64, 64, 32)     │           128 │
│ (BatchNormalization)            │                        │               │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ conv2d_1 (Conv2D)               │ (None, 64, 64, 32)     │         9,248 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ batch_normalization_1           │ (None, 64, 64, 32)     │           128 │
│ (BatchNormalization)            │                        │               │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ max_pooling2d (MaxPooling2D)    │ (None, 32, 32, 32)     │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dropout (Dropout)               │ (None, 32, 32, 32)     │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ conv2d_2 (Conv2D)               │ (None, 32, 32, 64)     │        18,496 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ batch_normalization_2           │ (None, 32, 32, 64)     │           256 │
│ (BatchNormalization)            │                        │               │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ conv2d_3 (Conv2D)               │ (None, 32, 32, 64)     │        36,928 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ batch_normalization_3           │ (None, 32, 32, 64)     │           256 │
│ (BatchNormalization)            │                        │               │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ max_pooling2d_1 (MaxPooling2D)  │ (None, 16, 16, 64)     │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dropout_1 (Dropout)             │ (None, 16, 16, 64)     │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ conv2d_4 (Conv2D)               │ (None, 16, 16, 128)    │        73,856 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ batch_normalization_4           │ (None, 16, 16, 128)    │           512 │
│ (BatchNormalization)            │                        │               │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ max_pooling2d_2 (MaxPooling2D)  │ (None, 8, 8, 128)      │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dropout_2 (Dropout)             │ (None, 8, 8, 128)      │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ flatten (Flatten)               │ (None, 8192)           │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense (Dense)                   │ (None, 256)            │     2,097,408 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ batch_normalization_5           │ (None, 256)            │         1,024 │
│ (BatchNormalization)            │                        │               │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dropout_3 (Dropout)             │ (None, 256)            │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense_1 (Dense)                 │ (None, 27)             │         6,939 │
└─────────────────────────────────┴────────────────────────┴───────────────┘

 Total params: 2,246,075 (8.57 MB)

 Trainable params: 2,244,923 (8.56 MB)

 Non-trainable params: 1,152 (4.50 KB)

In [6]:
# Entraînement
history = model.fit(
    train_generator,
    epochs=10,
    validation_data=validation_generator,
    callbacks=[checkpoint, reduce_lr, early_stopping]
)

# Évaluer sur les données de validation
test_loss, test_accuracy = model.evaluate(validation_generator)
print(f"Validation Accuracy: {test_accuracy * 100:.2f}%")

# Sauvegarder le modèle final
model.save("asl_model_final.keras")

/home/youness.ait-hadichou/Documents/5ETI/Projet_majeure/projet/lib/python3.10/site-packages/keras/src/trainers/data_adapters/py_dataset_adapter.py:121: UserWarning: Your `PyDataset` class should call `super().__init__(**kwargs)` in its constructor. `**kwargs` can include `workers`, `use_multiprocessing`, `max_queue_size`. Do not pass these arguments to `fit()`, as they will be ignored.
  self._warn_if_super_not_called()


Epoch 1/10
1013/1013 ━━━━━━━━━━━━━━━━━━━━ 1158s 1s/step - accuracy: 0.2936 - loss: 3.4229 - val_accuracy: 0.3888 - val_loss: 3.0078 - learning_rate: 5.0000e-04
Epoch 2/10
1013/1013 ━━━━━━━━━━━━━━━━━━━━ 1083s 1s/step - accuracy: 0.8015 - loss: 1.2587 - val_accuracy: 0.6682 - val_loss: 1.6499 - learning_rate: 5.0000e-04
Epoch 3/10
1013/1013 ━━━━━━━━━━━━━━━━━━━━ 1156s 1s/step - accuracy: 0.9104 - loss: 0.8064 - val_accuracy: 0.7815 - val_loss: 1.2207 - learning_rate: 5.0000e-04
Epoch 4/10
1013/1013 ━━━━━━━━━━━━━━━━━━━━ 1225s 1s/step - accuracy: 0.9416 - loss: 0.6504 - val_accuracy: 0.7717 - val_loss: 1.2353 - learning_rate: 5.0000e-04
Epoch 5/10
1013/1013 ━━━━━━━━━━━━━━━━━━━━ 1283s 1s/step - accuracy: 0.9491 - loss: 0.6059 - val_accuracy: 0.8290 - val_loss: 0.9986 - learning_rate: 5.0000e-04
Epoch 6/10
1013/1013 ━━━━━━━━━━━━━━━━━━━━ 1065s 1s/step - accuracy: 0.9564 - loss: 0.5627 - val_accuracy: 0.8183 - val_loss: 1.0350 - learning_rate: 5.0000e-04
Epoch 7/10
1013/1013 ━━━━━━━━━━━━━━━━━━━

In [ ]:
# Obtenez les prédictions du modèle
y_pred_probs = model.predict(validation_generator)  # Probabilités prédites
y_pred = np.argmax(y_pred_probs, axis=1)            # Classes prédites

# Obtenez les vraies étiquettes
y_true = validation_generator.classes

# Classes dans le générateur
class_labels = list(validation_generator.class_indices.keys())

# Rapport de classification
print("Classification Report:")
print(classification_report(y_true, y_pred, target_names=class_labels))

# Matrice de confusion
conf_matrix = confusion_matrix(y_true, y_pred)

# Visualisation de la matrice de confusion
plt.figure(figsize=(12, 10))
sns.heatmap(conf_matrix, annot=True, fmt="d", cmap="Blues", xticklabels=class_labels, yticklabels=class_labels)
plt.xlabel("Predicted Labels")
plt.ylabel("True Labels")
plt.title("Confusion Matrix")
plt.show()

In [ ]:
# Récupérer les données de l'historique
accuracy = history.history['accuracy']
val_accuracy = history.history['val_accuracy']
loss = history.history['loss']
val_loss = history.history['val_loss']

# Graphe de la précision
plt.figure(figsize=(12, 6))
plt.plot(accuracy, label="Training Accuracy")
plt.plot(val_accuracy, label="Validation Accuracy")
plt.title("Model Accuracy")
plt.xlabel("Epochs")
plt.ylabel("Accuracy")
plt.legend()
plt.grid()
plt.show()

# Graphe de la perte
plt.figure(figsize=(12, 6))
plt.plot(loss, label="Training Loss")
plt.plot(val_loss, label="Validation Loss")
plt.title("Model Loss")
plt.xlabel("Epochs")
plt.ylabel("Loss")
plt.legend()
plt.grid()
plt.show()

In [2]:
from tensorflow.keras.models import load_model
import cv2
import numpy as np

model = load_model("asl_model_final.keras")

target_classes = list("ABCDEFGHIJKLMNOPQRSTUVWXYZ") + ["nothing"]
class_indices = {v: k for k, v in enumerate(target_classes)}  

ROI_SIZE = (64, 64)

cap = cv2.VideoCapture(0)  

while True:
    ret, frame = cap.read()
    if not ret:
        print("Erreur lors de la capture de l'image")
        break

    height, width, _ = frame.shape
    roi_start_x = width // 2 - 100
    roi_start_y = height // 2 - 100
    roi_end_x = width // 2 + 100
    roi_end_y = height // 2 + 100

    cv2.rectangle(frame, (roi_start_x, roi_start_y), (roi_end_x, roi_end_y), (0, 255, 0), 2)

    roi = frame[roi_start_y:roi_end_y, roi_start_x:roi_end_x]
    roi_resized = cv2.resize(roi, ROI_SIZE) 
    roi_normalized = roi_resized / 255.0  
    input_frame = np.expand_dims(roi_normalized, axis=0)  

    predictions = model.predict(input_frame)
    predicted_class_index = np.argmax(predictions)
    predicted_class = target_classes[predicted_class_index]  

    cv2.putText(frame, f"Prediction: {predicted_class}", (10, 30), cv2.FONT_HERSHEY_SIMPLEX, 
                1, (0, 255, 0), 2, cv2.LINE_AA)

    cv2.imshow("Real-time ASL Recognition", frame)

    if cv2.waitKey(1) & 0xFF == ord('q'):
        break

cap.release()
cv2.destroyAllWindows()


2025-01-13 16:11:04.630207: I external/local_xla/xla/tsl/cuda/cudart_stub.cc:32] Could not find cuda drivers on your machine, GPU will not be used.
2025-01-13 16:11:04.636308: I external/local_xla/xla/tsl/cuda/cudart_stub.cc:32] Could not find cuda drivers on your machine, GPU will not be used.
2025-01-13 16:11:04.657404: E external/local_xla/xla/stream_executor/cuda/cuda_fft.cc:477] Unable to register cuFFT factory: Attempting to register factory for plugin cuFFT when one has already been registered
E0000 00:00:1736781064.688275   39303 cuda_dnn.cc:8310] Unable to register cuDNN factory: Attempting to register factory for plugin cuDNN when one has already been registered
E0000 00:00:1736781064.695487   39303 cuda_blas.cc:1418] Unable to register cuBLAS factory: Attempting to register factory for plugin cuBLAS when one has already been registered
2025-01-13 16:11:04.717490: I tensorflow/core/platform/cpu_feature_guard.cc:210] This TensorFlow binary is optimized to use available CPU ins

1/1 ━━━━━━━━━━━━━━━━━━━━ 1s 703ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 91ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 68ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 60ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 58ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 121ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 62ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 67ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 62ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 65ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 67ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 78ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 81ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 64ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 65ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 68ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 65ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 60ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 62ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 64ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 60ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 64ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 56ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 60ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 61ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 60ms/step
1/1 ━━━━━━